## Transfer Scene Viewing Eyetracking Files to Compute Canada (.ps1)

This should be kept and run in the folder where scene viewing eyetracking files are backed up. It will send new files (folders detected within 24 hours) to Compute Canada. Ensure that all 6 session folders match after sending.  
    - (4 Encoding, 3 Retrieval)

VERSION: `Scene_Eyetrack_Auto_Sync_General.ps1`

This script will promt you for username, pathway, and files. It will output filename and size for tracking. 

In [ ]:
$currentDate = Get-Date
$localFolders = Get-ChildItem -Path . -Directory

$transferredCount = 0
$transferredDirectories = @()

$username = Read-Host "Please enter your username"

$remotePathBase = "/home/$username/scratch/Actual_Final_SceneViewing_EyeTracking"

Write-Output "The constructed remote path is: $remotePathBase"
$pathConfirmation = Read-Host "Is this path correct? [Y/N]"

if ($pathConfirmation -ne "Y") {
    $remotePathBase = Read-Host "Please enter the full correct remote path"
}

foreach ($localFolder in $localFolders) {

    $recentDirectories = Get-ChildItem -Path $localFolder.FullName -Directory | Where-Object {
        ($currentDate - $_.CreationTime).TotalHours -le 24
    }

    foreach ($dir in $recentDirectories) {

        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        $creationTime = $dir.CreationTime
        $userInput = Read-Host "Do you want to transfer $($dir.Name) [Size: $($directorySize) MB, Created: $($creationTime)] to the server at `"$remotePathBase`"? [Y/N]"

        if ($userInput -eq "Y") {
            $remotePath = "$remotePathBase/$($localFolder.Name)/$($dir.Name)"

            scp -r $dir.FullName "$username@graham.computecanada.ca:`"$remotePath`""
            if ($?) {
                $transferredCount++
                $transferredDirectories += $dir.Name
                
                Write-Output ("Directory '{0}' transferred to '{1}'. Size: {2:N2} MB" -f $dir.Name, $remotePath, $directorySize)
            } else {
                Write-Warning "Failed to transfer directory: $dir"
            }
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}


VERSION: `Scene_Eyetrack_Auto_Sync_(name).ps1`

Replace remote pathbase and scp command to reflect the user and save with your name using Powershell ISE. Note that there will be no prompts as the script is designed for maximum efficiency so make sure you know what you are doing. 

In [ ]:
$currentDate = Get-Date
$localFolders = Get-ChildItem -Path . -Directory

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/FinalSceneViewingEyetracking/"

foreach ($localFolder in $localFolders) {

    $recentDirectories = Get-ChildItem -Path $localFolder.FullName -Directory | Where-Object {
        ($currentDate - $_.CreationTime).TotalHours -le 24
    }

    foreach ($dir in $recentDirectories) {

        $remotePath = "$remotePathBase/$($localFolder.Name)/$($dir.Name)"

        scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            
            $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
            Write-Output ("Directory '{0}' transferred to '{1}'. Size: {2:N2} MB" -f $dir.Name, $remotePath, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}
